In [1]:
import os
os.environ['HF_HOME'] = '/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/.hf_cache' ## THIS HAS TO BE BEFORE YOU IMPORT TRANSFORMERS

import transformers
import torch

from abc import ABC, abstractmethod
from pathlib import Path
from typing import Optional
from PIL import Image
from my_secrets import LLAMA3_INSTRUCT_ACCESS_TOKEN
from agent_utils import select_best_gpu

from pathology_detector import PathologyDetector, CheXagentVisionTransformerPathologyDetector
from pathology_sets import Pathologies

from phrase_grounder import PhraseGrounder, BioVilTPhraseGrounder

class GenerationEngine(ABC):


    @abstractmethod
    def generate_model_output(self, system_prompt: str , image_context_prompt: str, user_prompt:Optional[str] = None, image_path = None):
        pass

    
    def contextualise_model(image_path: Path, pathology_detector: PathologyDetector, phrase_grounder: Optional[PhraseGrounder] = None, examples = True) -> str:
        
        pathology_detection_threshold = 0.5

        if pathology_detector is not None:
            pathology_confidences = pathology_detector.detect_pathologies(image_path, threshold = pathology_detection_threshold)
            # print(pathology_confidences)
        else:
            return RuntimeError("Pathology detector not provided")
        
        #### PROMPT PIPELINE ###
        if len(pathology_confidences) == 0:
            system_prompt = """ You are a helpful assistant, specialising in radiology and interpreting Chest X-rays. However there is insufficient data make any comments. Just mention it is possible there are no findings and this should be double checked by a radiologist."""
            image_context_prompt = f"""No pathologies were detected in the chest X-ray image. The user will now interact with you."""
            return system_prompt, image_context_prompt
        

        system_prompt = """You are a helpful assistant, specialising in radiology and interpreting Chest X-rays. Please answer CONCISELY and professionally as a radiologist would. You should not be confident unless the data is confident. Use language that reflects the confidence of the data."""
        image_context_prompt_final_part = """
            This understanding is crucial for accurately processing and responding to queries on the chest X-ray analysis. Structure your answers based on confidence and pathologies.
            Double check before you submit your response to ensure you have factored in all the data and followed my instructions carefully.
            You will now interact with the user.
            """

        if phrase_grounder is None:
            image_context_prompt = f"""
                You are being provided with data derived from analyzing a chest X-ray, which includes findings on potential pathologies alongside their confidence levels.
                This information comes from a specialized diagnostic tools.
                It's important to recognize how to interpret this when responding to queries:

                Pathology Detection with Confidence Scores:
                {pathology_confidences}
                Please note the closer the value to 1, the more likely the pathology is present in the image. 

                It is important to factor medical knowledge and the specifics of each case, if supplied, into your responses.
                {image_context_prompt_final_part}
                """

        else:
            pathologies = [pathology for pathology, confidence in pathology_confidences.items() if confidence > pathology_detection_threshold]
            grounded_pathologies_confidences = phrase_grounder.get_pathology_lateral_position(image_path, pathologies)
            # print(grounded_pathologies_confidences)
            if len(grounded_pathologies_confidences) == 0:
                grounded_pathologies_confidences = "No lateral positions could be confidently determined for any pathologies detected."

            image_context_prompt = f"""
            You are being provided with data derived from analyzing a chest X-ray, which includes findings on potential pathologies alongside their confidence levels and, separately, possible lateral locations of these pathologies with their own confidence scores.
            This information comes from two specialized diagnostic tools.
            It's important to recognize how to interpret these datasets together when responding to queries:

            Pathology Detection with Confidence Scores:
            {pathology_confidences}

            Phrase Grounding Locations with Confidence Scores:
            {grounded_pathologies_confidences}

            This separate dataset provides potential lateral locations for some of the detected pathologies, each with its own confidence score, indicating the model's certainty about each pathology's location.
            For instance, left Pleural Effusion is listed with a confidence score of 0.53, suggesting the location of Pleural Effusion to be on the left side with moderate confidence.

            When you interact with end users, remember:

            - A pathology and its lateral location (e.g., Pleural Effusion and left Pleural Effusion) are part of the same finding. The location attribute is an additional detail about where the pathology is likely found, not an indicator of a separate pathology. 
            - Synthesize the pathology detection and localization data. DO NOT TALK ABOUT THEM SEPERATELY. {"Here is a model example, 'Highly likely there is Pleural Effusion (detection confidence: 0.80), and it is possibly on the left side (localisation confidence: 0.53).'" if examples else ""}
            - Confidence scores from the pathology detection and phrase grounding tools are not directly comparable. They serve as indicators of confidence within their respective contexts of pathology detection and localisation.
            - A missing lateral location does not imply the absence of a pathology; it indicates the localisation could not be confidently determined.
            - If there is any discrepancy between the pathology detection and phrase grounding tools, detection data takes precedence as it more reliably identifies pathologies.

            It is important to factor medical knowledge and the specifics of each case, if supplied, into your responses. For example, pathologies located on both sides are called bilateral. Heart related observations are usually on the left/ middle.
            {image_context_prompt_final_part}
            """
            
        return system_prompt, image_context_prompt



class Llama3Generation(GenerationEngine):

    def __init__(self):
        self.model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

        self.pipeline = transformers.pipeline(
            "text-generation",
            model=self.model_id,
            model_kwargs={"torch_dtype": torch.bfloat16},
            device_map= select_best_gpu() ,
            token=LLAMA3_INSTRUCT_ACCESS_TOKEN,
        )

    def generate_model_output(self, system_prompt: str , image_context_prompt: str, user_prompt:Optional[str] = None):
        def format_output(output_text: str) -> str:
            return "\n".join(output_text.split(". "))  # print output text with each sentence on a new line

        terminators = [
            self.pipeline.tokenizer.eos_token_id,
            self.pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        if user_prompt is not None:
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": image_context_prompt +"\n" + user_prompt},
            ]

            prompt = self.pipeline.tokenizer.apply_chat_template(
                    messages, 
                    tokenize=False, 
                    add_generation_prompt=True
            )

            outputs = self.pipeline(
                prompt,
                max_new_tokens=512,
                eos_token_id=terminators,
                do_sample=True,
                temperature=0.6,
                top_p=0.9,
            )
            output_text = outputs[0]["generated_text"][len(prompt):]
            print(format_output(output_text))
            return outputs[0]["generated_text"][len(prompt):]
        
        else:
            # setup chat loop
            chat_history = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": image_context_prompt},
            ]

            user_prompt = ""
            while user_prompt != "exit":
                prompt = self.pipeline.tokenizer.apply_chat_template(
                    chat_history, 
                    tokenize=False, 
                    add_generation_prompt=True
                )

                outputs = self.pipeline(
                    prompt,
                    max_new_tokens=512,
                    eos_token_id=terminators,
                    do_sample=True,
                    temperature=0.6,
                    top_p=0.9,
                )

                output_text = outputs[0]["generated_text"][len(prompt):]
                chat_history.append({"role": "assistant", "content": output_text})
                print(format_output(output_text))

                user_prompt = input("User: ")
                print(f"\n{user_prompt}")
                chat_history.append({"role": "user", "content": user_prompt})
            return chat_history

class CheXagentLanguageModelGeneration(GenerationEngine):

    def __init__(self, processor, model, generation_config, device, dtype):
        self.processor = processor
        self.model = model
        self.generation_config = generation_config
        self.device = device
        self.dtype = dtype

    def generate_model_output(self, system_prompt: str , image_context_prompt: str, user_prompt:Optional[str] = None):
        if user_prompt is None:
            return RuntimeError("User prompt not provided - Chat mode CURRENTLY not supported with CheXagent Language Model")
        inputs = self.processor(
        images=None, text=f"[INST]{image_context_prompt}[/INST] USER: <s>{user_prompt} ASSISTANT: <s>", return_tensors="pt"
        ).to(device=self.device)
        output = self.model.generate(**inputs, generation_config=self.generation_config,language_only = True)[0]
        response = self.processor.tokenizer.decode(output, skip_special_tokens=True)
        print(response)
        return response
    

class CheXagentEndToEndGeneration(GenerationEngine):

    def __init__(self, processor, model, generation_config, device, dtype):
        self.processor = processor
        self.model = model
        self.generation_config = generation_config
        self.device = device
        self.dtype = dtype

    def generate_model_output(self, system_prompt: str , image_context_prompt: str, user_prompt:Optional[str],image_path: Path):
        images = [Image.open(image_path).convert("RGB")]
        inputs = self.processor(
            images=images, text=f" USER: <s>{user_prompt} ASSISTANT: <s>", return_tensors="pt"
        ).to(device=self.device, dtype=self.dtype)
        # print(inputs.keys())
        output = self.model.generate(**inputs, generation_config=self.generation_config)[0]
        response = self.processor.tokenizer.decode(output, skip_special_tokens=True)
        print(response)
        return response


In [2]:
pathology_detector = CheXagentVisionTransformerPathologyDetector(pathologies=Pathologies.CHEXPERT)
phrase_grounder = BioVilTPhraseGrounder(detection_threshold=0.5)
l3 = Llama3Generation()
cheXagent_lm = CheXagentLanguageModelGeneration(pathology_detector.processor, pathology_detector.model, pathology_detector.generation_config, pathology_detector.device, pathology_detector.dtype)
cheXagent_e2e = CheXagentEndToEndGeneration(pathology_detector.processor, pathology_detector.model, pathology_detector.generation_config, pathology_detector.device, pathology_detector.dtype)

GPU 0: NVIDIA RTX 6000 Ada Generation, Free memory: 46315 MB
GPU 1: NVIDIA RTX 6000 Ada Generation, Free memory: 48643 MB
Selecting GPU 1 with 48643 MB free memory, Device = cuda:1


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

CheXagent Model loaded


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'CXRBertTokenizer'.
You are using a model of type bert to instantiate a model of type cxr-bert. This is not supported for all configurations of models and can yield errors.
/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/.venv/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at microsoft/BiomedVLP-BioViL-T were not used when initializing CXRBertModel: ['be

Using downloaded and verified file: /tmp/biovil_t_image_model_proj_size_128.pt
GPU 0: NVIDIA RTX 6000 Ada Generation, Free memory: 45885 MB
GPU 1: NVIDIA RTX 6000 Ada Generation, Free memory: 31391 MB
Selecting GPU 0 with 45885 MB free memory, Device = cuda:0
GPU 0: NVIDIA RTX 6000 Ada Generation, Free memory: 45313 MB
GPU 1: NVIDIA RTX 6000 Ada Generation, Free memory: 31391 MB
Selecting GPU 0 with 45313 MB free memory, Device = cuda:0


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
chexpert_test_csv_path = Path("/vol/biodata/data/chest_xray/CheXpert-v1.0-small/CheXpert-v1.0-small/test.csv")
chexpert_test_path = Path("/vol/biomedic3/bglocker/ugproj2324/nns20/datasets/CheXpert/small/")

with open(chexpert_test_csv_path, 'r') as f:
    lines = f.readlines()
    header = lines[0].split(",")[1:]
    # print(header)
    for i, line in enumerate(lines[40:]):
        if i % 1000 == 0:
            print(f"Collecting image {i}")

        image_path = line.split(",")[0]
        print(f"{image_path=}")
        image_path = chexpert_test_path / image_path
        system_prompt, image_context_prompt = GenerationEngine.contextualise_model(image_path=image_path,pathology_detector=pathology_detector, phrase_grounder=phrase_grounder, examples=False)
        # print(system_prompt)
        # print(image_context_prompt)

        user_prompt = f"""Write a report on the chest x-ray image."""
        print("L3 agent:")
        l3.generate_model_output(system_prompt, image_context_prompt,user_prompt=user_prompt)
        print("CheXagent agent:")
        cheXagent_lm.generate_model_output(system_prompt, image_context_prompt,user_prompt=user_prompt)
        print("CheXagent:")
        cheXagent_e2e.generate_model_output(system_prompt, image_context_prompt,user_prompt=user_prompt, image_path=image_path)
        print(f" -----------------")
        if i == 3: 
            break


/vol/biomedic3/bglocker/ugproj2324/nns20/cxr-agent/.hf_cache/modules/transformers_modules/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


image_path='test/patient64765/study1/view1_frontal.jpg'
L3 agent:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Based on the chest X-ray image provided, I did not detect any obvious pathologies or abnormalities.

It is possible that the image may not have captured any significant findings, and I would recommend that a radiologist review the image to confirm the absence of any notable features
Additionally, the image may not have been taken with the correct positioning or technique, which could also impact the accuracy of the interpretation.

In the absence of any obvious findings, I would recommend that the patient's clinical history and any relevant symptoms be taken into consideration to determine if further imaging or evaluation is necessary.

Please note that this is a preliminary report and should not be used as a definitive diagnosis without further review and confirmation by a radiologist.
CheXagent agent:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

L3 agent:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Based on the analysis of the chest X-ray image, the following findings are reported:

There is moderate confidence (0.91) in the presence of Lung Opacity
Additionally, there is high confidence (0.97) in the presence of Pleural Effusion
The location of this Pleural Effusion is likely on the left side, with a confidence score of 0.53.

There is also moderate confidence (0.54) in the presence of Atelectasis
However, the location of this Atelectasis could not be confidently determined.

Lastly, there is high confidence (0.87) in the presence of Support Devices, which are likely located on both the left and right sides, with confidence scores of 0.54 and 0.6, respectively.

In summary, the chest X-ray image shows Lung Opacity with moderate confidence, Pleural Effusion with high confidence and likely located on the left side, Atelectasis with moderate confidence but uncertain location, and Support Devices with high confidence and likely located bilaterally.
CheXagent agent:


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The chest x-ray image shows several pathologies, including Lung Opacity, Atelectasis, Pleural Effusion, and Support Devices. The Support Devices are located on the left and right sides with moderate and high confidence, respectively.
CheXagent:
In comparison with the study of ___, the monitoring and support devices are essentially unchanged. Continued enlargement of the cardiac silhouette with mild elevation of pulmonary venous pressure. Retrocardiac opacification is consistent with volume loss in the left lower lobe and small pleural effusion.
 -----------------
